In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
import pylab as pl
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)

## Specifiy dementia subtype

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

## Specify exposure window

In [6]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

## Load relevant dataframe and create intercept

In [7]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_' + subtype + '_' + window +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [8]:

pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'non_insomnia_GP_consultations', 'stroke',
       'non_stroke_vascular_disease', 'hypertension', 'diabetes',
       'mental_illness_non_smi', 'mental_illness_smi', 'sleep_apnoea',
       'chronic_pulmonary_disease', 'epilepsy', 'age_at_index_date:65-69',
       'age_at_index_date:70-74', 'age_at_index_date:75-79',
       'age_at_index_date:80-84', 'age_at_index_date:85-89',
       'age_at_index_date:90-99', 'age_at_index_date:above_99',
       'insomnia_count:0', 'insomnia_count:above_10',
       'non_insomnia_GP_consultations:0', 'insomnia_count:1_5', 'insomnia',
       'insomnia_count:6_10', 'benzo_and_z_drugs_any', 'insomnia_any',
       'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000',
 

In [9]:
pt_features['intercept'] = 1.0

In [10]:
# pt_features.sort_values(by='insomnia_count<=10',ascending=False)

In [11]:
columns_always_for_inclusion = [
    'female',
    'non_insomnia_GP_consultations', 'stroke',
    'non_stroke_vascular_disease', 'hypertension',
    'diabetes', 'mental_illness_non_smi', 'mental_illness_smi',
    'sleep_apnoea', 'chronic_pulmonary_disease', 'epilepsy',
    'mood_stabilisers_pdds',
    'other_sedatives_pdds', 'antidepressants_pdds', 
    'depot_antipsychotics_pdds', 'antipsychotics_pdds',
    'benzo_and_z_drugs_pdds'
]

## Insomnia consultations

### insomnia yes/no

In [12]:
cols_for_inclusion_here = ['insomnia_any','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

The following variables are being removed as mean = 0:



*Stage 1*
Univariate results
                               Univariate OR  p value  [0.025  0.975]
benzo_and_z_drugs_100_pdds             0.980    0.601   0.908   1.058
chronic_pulmonary_disease              1.075    0.024   1.010   1.145
diabetes                               1.210    0.000   1.107   1.322
epilepsy                               1.567    0.000   1.246   1.970
female                                 1.000    1.000   0.967   1.034
hypertension                           0.954    0.022   0.916   0.993
insomnia_any                           1.295    0.000   1.200   1.397
mental_illness_non_smi                 1.427    0.000   1.356   1.502
mental_illness_smi                     1.560    0.000   1.372   1.774
non_insomnia_GP_consultations          1.001    0.000   1.001   1.001
non_stroke_vascular_disease            1.102    0.000   1.044   1.163
sleep_apnoea                           1.118    0.739   0.581   2.150
str

In [13]:
univariate_and_multivariate_results

,Univariate OR,p value,[0.025,0.975],Multivariate OR,p value,[0.025,0.975]
benzo_and_z_drugs_100_pdds,0.980,0.601,0.908,1.058,0.968,0.423,0.893,1.048
diabetes,1.210,0.000,1.107,1.322,1.194,0.000,1.086,1.312
epilepsy,1.567,0.000,1.246,1.970,1.378,0.007,1.092,1.739
female,1.000,1.000,0.967,1.034,0.890,0.000,0.853,0.930
hypertension,0.954,0.022,0.916,0.993,0.836,0.000,0.794,0.881
insomnia_any,1.295,0.000,1.200,1.397,1.233,0.000,1.137,1.337
mental_illness_non_smi,1.427,0.000,1.356,1.502,1.511,0.000,1.419,1.608
mental_illness_smi,1.560,0.000,1.372,1.774,1.148,0.051,1.000,1.319
stroke,1.338,0.000,1.219,1.469,1.292,0.000,1.172,1.426


### insomnia (continuous variable)

In [14]:
cols_for_inclusion_here = ['insomnia','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

The following variables are being removed as mean = 0:



*Stage 1*
Univariate results
                               Univariate OR  p value  [0.025  0.975]
benzo_and_z_drugs_100_pdds             0.980    0.601   0.908   1.058
chronic_pulmonary_disease              1.075    0.024   1.010   1.145
diabetes                               1.210    0.000   1.107   1.322
epilepsy                               1.567    0.000   1.246   1.970
female                                 1.000    1.000   0.967   1.034
hypertension                           0.954    0.022   0.916   0.993
insomnia                               1.126    0.000   1.085   1.168
mental_illness_non_smi                 1.427    0.000   1.356   1.502
mental_illness_smi                     1.560    0.000   1.372   1.774
non_insomnia_GP_consultations          1.001    0.000   1.001   1.001
non_stroke_vascular_disease            1.102    0.000   1.044   1.163
sleep_apnoea                           1.118    0.739   0.581   2.150
str

In [15]:
univariate_and_multivariate_results

,Univariate OR,p value,[0.025,0.975],Multivariate OR,p value,[0.025,0.975]
benzo_and_z_drugs_100_pdds,0.980,0.601,0.908,1.058,0.967,0.419,0.893,1.048
diabetes,1.210,0.000,1.107,1.322,1.194,0.000,1.087,1.312
epilepsy,1.567,0.000,1.246,1.970,1.382,0.006,1.095,1.744
female,1.000,1.000,0.967,1.034,0.895,0.000,0.857,0.934
hypertension,0.954,0.022,0.916,0.993,0.838,0.000,0.796,0.882
insomnia,1.126,0.000,1.085,1.168,1.099,0.000,1.058,1.141
mental_illness_non_smi,1.427,0.000,1.356,1.502,1.513,0.000,1.421,1.611
mental_illness_smi,1.560,0.000,1.372,1.774,1.147,0.053,0.998,1.317
stroke,1.338,0.000,1.219,1.469,1.297,0.000,1.176,1.430


### insomnia quantiles

In [16]:
cols_for_inclusion_here = ['benzo_and_z_drugs_100_pdds','insomnia_count:1_5','insomnia_count:6_10','insomnia_count:above_10']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

The following variables are being removed as mean = 0:



*Stage 1*
Univariate results
                               Univariate OR  p value  [0.025  0.975]
benzo_and_z_drugs_100_pdds             0.980    0.601   0.908   1.058
chronic_pulmonary_disease              1.075    0.024   1.010   1.145
diabetes                               1.210    0.000   1.107   1.322
epilepsy                               1.567    0.000   1.246   1.970
female                                 1.000    1.000   0.967   1.034
hypertension                           0.954    0.022   0.916   0.993
insomnia_count:1_5                     1.285    0.000   1.190   1.387
insomnia_count:6_10                    2.750    0.014   1.224   6.177
insomnia_count:above_10                1.286    0.618   0.479   3.452
mental_illness_non_smi                 1.427    0.000   1.356   1.502
mental_illness_smi                     1.560    0.000   1.372   1.774
non_insomnia_GP_consultations          1.001    0.000   1.001   1.001
non

In [17]:
univariate_and_multivariate_results

,Univariate OR,p value,[0.025,0.975],Multivariate OR,p value,[0.025,0.975]
benzo_and_z_drugs_100_pdds,0.980,0.601,0.908,1.058,0.968,0.424,0.894,1.048
diabetes,1.210,0.000,1.107,1.322,1.194,0.000,1.086,1.312
epilepsy,1.567,0.000,1.246,1.970,1.377,0.007,1.091,1.738
female,1.000,1.000,0.967,1.034,0.890,0.000,0.853,0.930
hypertension,0.954,0.022,0.916,0.993,0.836,0.000,0.794,0.881
insomnia_count:1_5,1.285,0.000,1.190,1.387,1.227,0.000,1.130,1.331
insomnia_count:6_10,2.750,0.014,1.224,6.177,2.307,0.045,1.020,5.214
insomnia_count:above_10,1.286,0.618,0.479,3.452,1.033,0.949,0.381,2.798
mental_illness_non_smi,1.427,0.000,1.356,1.502,1.509,0.000,1.417,1.607
mental_illness_smi,1.560,0.000,1.372,1.774,1.149,0.049,1.000,1.320


## Benzo and z-drug PDDs

In [18]:
### benzos yes/no

In [19]:
# cols_for_inclusion_here = ['benzo_and_z_drugs_any']
# training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
# summary_table,result = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

In [20]:
# result

### benzo PDDs

In [21]:
# cols_for_inclusion_here = ['benzo_and_z_drugs_pdds']
# training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
# summary_table,result = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

In [22]:
# result

### benzos quantiles

In [23]:
# cols_for_inclusion_here = ['benzo_and_z_drugs_pdds:1_10', 'benzo_and_z_drugs_pdds:11_100',
#        'benzo_and_z_drugs_pdds:101_1000', 'benzo_and_z_drugs_pdds:1001_10000',
#        'benzo_and_z_drugs_pdds:above_10000']
# training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
# summary_table,result = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

In [24]:
# result